In [158]:
import pandas as pd
import numpy as np
import random 

import statsmodels.formula.api as sm
import sys
from matplotlib import pyplot as plt
%matplotlib inline 

sys.version

'3.6.3 |Anaconda custom (64-bit)| (default, Oct  6 2017, 12:04:38) \n[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]'

In [197]:
### question: can a model discover with 36 obersvations if the signs make a difference?

### case 1: binary model, treatment/no treatment
### the blue sign always 
### yields a 200 gram difference

In [135]:
### get locations

# 0 = gym
# 1 = medical
# 2 = retail
# 3 = work/lobbies
location_type = [0, 1, 2, 3]
location = np.random.choice(location_type, 36, p=[0.25, 0.1, 0.25, 0.4])
location 

array([3, 3, 3, 3, 3, 3, 2, 3, 3, 2, 3, 2, 2, 3, 1, 3, 3, 0, 1, 1, 0, 3, 3,
       0, 1, 2, 2, 3, 3, 3, 2, 3, 0, 2, 2, 3])

In [136]:
treatment_type = [0,2]
treatments = np.random.choice(treatment_type, 36)
treatments

array([0, 2, 2, 0, 2, 2, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0,
       0, 2, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 0])

In [137]:
a = [x for x in range(60,721,5)]
duration = np.random.choice(a, 36)
duration

array([400, 215, 155, 295, 660, 520, 520, 505, 530,  60, 410, 545, 280,
       545, 595, 190,  70,  65, 625, 365, 265, 570, 640, 275, 285, 370,
       580, 145, 135, 550, 325, 610, 705, 140,  75, 675])

In [138]:
start = np.array([abs(np.random.normal(loc=5,size=36))*1000])
start

array([[ 4808.31537349,  7001.45808038,  4611.28568215,  4042.75768098,
         6142.31277737,  5258.73985827,  4899.37512682,  3502.0741862 ,
         4310.77949605,  4979.52374228,  3569.82113093,  4534.41917639,
         5690.92086643,  3951.21553013,  3958.01168666,  5418.93045637,
         4503.65261521,  5493.03201681,  4792.86108325,  5233.6555822 ,
         5746.24626698,  4127.52824691,  7404.95939124,  5673.87387884,
         4670.33523337,  4801.17956445,  6126.16936154,  4537.13696389,
         7198.80547791,  4168.54258067,  5189.41888614,  5808.63867543,
         5203.18898363,  6517.4417516 ,  3458.0928765 ,  6036.60825651]])

In [139]:
diff = abs(np.random.normal(loc = 0.5, size=36))*100
diff

array([ 191.49098115,   42.97773165,   31.19332982,   65.6642781 ,
          3.61665465,  167.79710432,   65.70581313,   94.92781505,
        146.61817789,    3.97885203,   81.68459268,   19.04427999,
         21.27379087,   80.42389817,   33.23115743,  213.71515105,
         54.1866892 ,  161.44762702,   85.3664048 ,  135.9263544 ,
         14.97328986,   39.86687294,   57.32014058,   47.80503731,
         98.6348173 ,  147.34278081,    4.83898473,   39.15219881,
         43.41603161,   73.82028964,   18.79799088,   47.93739425,
        100.97703139,  234.16845438,  142.25445838,  102.6447944 ])

In [140]:
### edit the diff so that anytime treatments == 2, the actual diff is overwritten and becomes 200 grams
diff_overwrite = []
for i, x in enumerate(diff):
    if treatments[i] > 0:
        diff_overwrite.append(200)
    else:
        diff_overwrite.append(x)

diff_overwrite

[191.4909811495624,
 200,
 200,
 65.66427810238504,
 200,
 200,
 65.705813127161278,
 94.927815053277584,
 200,
 200,
 200,
 19.044279994140222,
 21.273790869353761,
 80.423898168970993,
 33.23115742572076,
 213.71515105359481,
 200,
 200,
 85.366404802516783,
 200,
 14.973289863550427,
 39.866872941754217,
 57.320140583826188,
 47.80503730862857,
 200,
 147.34278080699448,
 200,
 39.152198814797536,
 43.416031609649636,
 73.820289644178899,
 200,
 200,
 200,
 200,
 200,
 102.64479439547642]

In [141]:
df_1 = pd.DataFrame(data = [location, treatments, duration, start[0], end[0], diff_overwrite]).T
df_1.columns = ['location', 'treatments', 'duration', 'start_wt', 'end_wt', 'wt_diff']
df_1.head(10)

,location,treatments,duration,start_wt,end_wt,wt_diff
0,3.0,0.0,400.0,4808.315373,3901.469582,191.490981
1,3.0,2.0,215.0,7001.458080,6299.199184,200.000000
2,3.0,2.0,155.0,4611.285682,3760.231332,200.000000
3,3.0,0.0,295.0,4042.757681,6009.467191,65.664278
4,3.0,2.0,660.0,6142.312777,4014.181749,200.000000
5,3.0,2.0,520.0,5258.739858,5998.739710,200.000000
6,2.0,0.0,520.0,4899.375127,6661.437990,65.705813
7,3.0,0.0,505.0,3502.074186,7723.771028,94.927815
8,3.0,2.0,530.0,4310.779496,4683.359724,200.000000
9,2.0,2.0,60.0,4979.523742,4250.605268,200.000000


In [143]:
### treatments
treat_dum = pd.get_dummies(df_1['treatments'])
treat_dum.head()
treat_dum.columns = ['control', 'blue']
treat_dum.head()

,control,blue
0,1,0
1,0,1
2,0,1
3,1,0
4,0,1


In [144]:
revised_df = pd.concat([df_1,treat_dum], axis=1)

In [157]:
revised_df

,location,treatments,duration,start_wt,wt_diff,control,blue
0,3.0,0.0,400.0,4808.315373,191.490981,1,0
1,3.0,2.0,215.0,7001.458080,200.000000,0,1
2,3.0,2.0,155.0,4611.285682,200.000000,0,1
3,3.0,0.0,295.0,4042.757681,65.664278,1,0
4,3.0,2.0,660.0,6142.312777,200.000000,0,1
5,3.0,2.0,520.0,5258.739858,200.000000,0,1
6,2.0,0.0,520.0,4899.375127,65.705813,1,0
7,3.0,0.0,505.0,3502.074186,94.927815,1,0
8,3.0,2.0,530.0,4310.779496,200.000000,0,1
9,2.0,2.0,60.0,4979.523742,200.000000,0,1


In [163]:
ols_revised = sm.ols(formula="wt_diff ~ blue", data=revised_df).fit(cov_type='cluster',
                                                         cov_kwds={'groups': revised_df['location']},
                                                         use_t=True)
ols_revised.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                wt_diff   R-squared:                       0.714
Model:                            OLS   Adj. R-squared:                  0.706
Method:                 Least Squares   F-statistic:                     100.8
Date:                Sun, 04 Mar 2018   Prob (F-statistic):            0.00210
Time:                        13:36:31   Log-Likelihood:                -183.20
No. Observations:                  36   AIC:                             370.4
Df Residuals:                      34   BIC:                             373.6
Df Model:                           1                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     75.6413     12.387      6.107      0.009      36.222     115.061
blue         124.3587     12.387     10.040      0.002      84.939     163.778
==============================================================================
Omnibus:                       24.636   Durbin-Watson:                   1.704
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               44.148
Skew:                           1.739   Prob(JB):                     2.59e-10
Kurtosis:                       7.163   Cond. No.                         2.56
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
"""

In [182]:
### case 2
### add back in red and blue,
### but make it so only blue is important 
treatment_type = [0,1,2]
treatments = np.random.choice(treatment_type, 36)
treatments

array([2, 2, 0, 2, 0, 1, 1, 2, 1, 0, 1, 0, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 0,
       0, 1, 2, 0, 1, 0, 0, 1, 1, 1, 1, 2, 0])

In [183]:
### get locations

# 0 = gym
# 1 = medical
# 2 = retail
# 3 = work/lobbies
location_type = [0, 1, 2, 3]
location = np.random.choice(location_type, 36, p=[0.25, 0.1, 0.25, 0.4])
location 

array([3, 3, 0, 0, 0, 2, 2, 2, 3, 0, 3, 1, 3, 3, 0, 3, 2, 3, 0, 2, 2, 3, 0,
       0, 2, 3, 0, 0, 3, 3, 2, 0, 3, 3, 0, 2])

In [186]:
treat_dum = pd.get_dummies(treatments)
treat_dum.head()
treat_dum.columns = ['control','red', 'blue']
treat_dum.head()

,control,red,blue
0,0,0,1
1,0,0,1
2,1,0,0
3,0,0,1
4,1,0,0


In [187]:
diff_overwrite = []
for i, x in enumerate(diff):
    if treatments[i] > 1:
        diff_overwrite.append(200)
    else:
        diff_overwrite.append(x)

diff_overwrite

[200,
 200,
 31.193329820572522,
 200,
 3.6166546494042717,
 167.79710431686027,
 65.705813127161278,
 200,
 146.61817788711571,
 3.9788520315045259,
 81.684592676218656,
 19.044279994140222,
 21.273790869353761,
 80.423898168970993,
 200,
 200,
 54.186689197869775,
 161.44762701990209,
 200,
 200,
 14.973289863550427,
 39.866872941754217,
 57.320140583826188,
 47.80503730862857,
 98.634817304728301,
 200,
 4.8389847316493428,
 39.152198814797536,
 43.416031609649636,
 73.820289644178899,
 18.797990875180254,
 47.937394254668895,
 100.97703138969271,
 234.16845437808442,
 200,
 102.64479439547642]

In [188]:
df_blue = pd.DataFrame(data = [location, treatments, duration, start[0], end[0], diff_overwrite]).T
df_blue.columns = ['location', 'treatments', 'duration', 'start_wt', 'wt_diff']
df_blue.head(10)
#df_two_treatments = pd.concat()

,location,treatments,duration,start_wt,end_wt,wt_diff
0,3.0,2.0,400.0,4808.315373,3901.469582,200.000000
1,3.0,2.0,215.0,7001.458080,6299.199184,200.000000
2,0.0,0.0,155.0,4611.285682,3760.231332,31.193330
3,0.0,2.0,295.0,4042.757681,6009.467191,200.000000
4,0.0,0.0,660.0,6142.312777,4014.181749,3.616655
5,2.0,1.0,520.0,5258.739858,5998.739710,167.797104
6,2.0,1.0,520.0,4899.375127,6661.437990,65.705813
7,2.0,2.0,505.0,3502.074186,7723.771028,200.000000
8,3.0,1.0,530.0,4310.779496,4683.359724,146.618178
9,0.0,0.0,60.0,4979.523742,4250.605268,3.978852


In [189]:
revised_blue = pd.concat([df_blue, treat_dum], axis =1)
revised_blue.head(10)

,location,treatments,duration,start_wt,end_wt,wt_diff,control,red,blue
0,3.0,2.0,400.0,4808.315373,3901.469582,200.000000,0,0,1
1,3.0,2.0,215.0,7001.458080,6299.199184,200.000000,0,0,1
2,0.0,0.0,155.0,4611.285682,3760.231332,31.193330,1,0,0
3,0.0,2.0,295.0,4042.757681,6009.467191,200.000000,0,0,1
4,0.0,0.0,660.0,6142.312777,4014.181749,3.616655,1,0,0
5,2.0,1.0,520.0,5258.739858,5998.739710,167.797104,0,1,0
6,2.0,1.0,520.0,4899.375127,6661.437990,65.705813,0,1,0
7,2.0,2.0,505.0,3502.074186,7723.771028,200.000000,0,0,1
8,3.0,1.0,530.0,4310.779496,4683.359724,146.618178,0,1,0
9,0.0,0.0,60.0,4979.523742,4250.605268,3.978852,1,0,0


In [191]:
ols_revised_blue = sm.ols(formula="wt_diff ~ blue + red", data=revised_blue).fit(cov_type='cluster',
                                                         cov_kwds={'groups': revised_blue['location']},
                                                         use_t=True)
ols_revised_blue.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                wt_diff   R-squared:                       0.669
Model:                            OLS   Adj. R-squared:                  0.649
Method:                 Least Squares   F-statistic:                     75.78
Date:                Sun, 04 Mar 2018   Prob (F-statistic):            0.00270
Time:                        13:49:08   Log-Likelihood:                -187.18
No. Observations:                  36   AIC:                             380.4
Df Residuals:                      33   BIC:                             385.1
Df Model:                           2                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     38.7678     13.599      2.851      0.065      -4.509      82.045
blue         161.2322     13.599     11.856      0.001     117.955     204.509
red           47.0850     17.562      2.681      0.075      -8.804     102.974
==============================================================================
Omnibus:                       13.022   Durbin-Watson:                   1.973
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               14.072
Skew:                           1.132   Prob(JB):                     0.000880
Kurtosis:                       5.064   Cond. No.                         4.11
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
"""

In [198]:
### case 3:  red and blue both have an effect
diff_overwrite = []
for i, x in enumerate(diff):
    if treatments[i] ==2:
        diff_overwrite.append(200)
    elif treatments[i] ==1:
        diff_overwrite.append(150)
    else:
        diff_overwrite.append(x)

diff_overwrite

[200,
 200,
 31.193329820572522,
 200,
 3.6166546494042717,
 150,
 150,
 200,
 150,
 3.9788520315045259,
 150,
 19.044279994140222,
 150,
 150,
 200,
 200,
 150,
 150,
 200,
 200,
 150,
 150,
 57.320140583826188,
 47.80503730862857,
 150,
 200,
 4.8389847316493428,
 150,
 43.416031609649636,
 73.820289644178899,
 150,
 150,
 150,
 150,
 200,
 102.64479439547642]

In [199]:
treat_dum = pd.get_dummies(treatments)
treat_dum.head()
treat_dum.columns = ['control','red', 'blue']
treat_dum.head()

,control,red,blue
0,0,0,1
1,0,0,1
2,1,0,0
3,0,0,1
4,1,0,0


In [201]:
df_red_blue = pd.DataFrame(data = [location, treatments, duration, start[0], diff_overwrite]).T
df_red_blue.columns = ['location', 'treatments', 'duration', 'start_wt', 'wt_diff']
df_red_blue.head(10)

,location,treatments,duration,start_wt,wt_diff
0,3.0,2.0,400.0,4808.315373,200.000000
1,3.0,2.0,215.0,7001.458080,200.000000
2,0.0,0.0,155.0,4611.285682,31.193330
3,0.0,2.0,295.0,4042.757681,200.000000
4,0.0,0.0,660.0,6142.312777,3.616655
5,2.0,1.0,520.0,5258.739858,150.000000
6,2.0,1.0,520.0,4899.375127,150.000000
7,2.0,2.0,505.0,3502.074186,200.000000
8,3.0,1.0,530.0,4310.779496,150.000000
9,0.0,0.0,60.0,4979.523742,3.978852


In [202]:
revised_red_blue = pd.concat([df_red_blue, treat_dum], axis =1)
revised_red_blue.head(10)

,location,treatments,duration,start_wt,wt_diff,control,red,blue
0,3.0,2.0,400.0,4808.315373,200.000000,0,0,1
1,3.0,2.0,215.0,7001.458080,200.000000,0,0,1
2,0.0,0.0,155.0,4611.285682,31.193330,1,0,0
3,0.0,2.0,295.0,4042.757681,200.000000,0,0,1
4,0.0,0.0,660.0,6142.312777,3.616655,1,0,0
5,2.0,1.0,520.0,5258.739858,150.000000,0,1,0
6,2.0,1.0,520.0,4899.375127,150.000000,0,1,0
7,2.0,2.0,505.0,3502.074186,200.000000,0,0,1
8,3.0,1.0,530.0,4310.779496,150.000000,0,1,0
9,0.0,0.0,60.0,4979.523742,3.978852,1,0,0


In [203]:
ols_revised_red_blue = sm.ols(formula="wt_diff ~ blue + red", data=revised_red_blue).fit(cov_type='cluster',
                                                         cov_kwds={'groups': revised_red_blue['location']},
                                                         use_t=True)
ols_revised_red_blue.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                wt_diff   R-squared:                       0.934
Model:                            OLS   Adj. R-squared:                  0.930
Method:                 Least Squares   F-statistic:                -2.644e+16
Date:                Sun, 04 Mar 2018   Prob (F-statistic):               1.00
Time:                        13:58:58   Log-Likelihood:                -152.00
No. Observations:                  36   AIC:                             310.0
Df Residuals:                      33   BIC:                             314.8
Df Model:                           2                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     38.7678     13.599      2.851      0.065      -4.509      82.045
blue         161.2322     13.599     11.856      0.001     117.955     204.509
red          111.2322     13.599      8.180      0.004      67.955     154.509
==============================================================================
Omnibus:                       20.191   Durbin-Watson:                   1.516
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               53.159
Skew:                           1.102   Prob(JB):                     2.86e-12
Kurtosis:                       8.530   Cond. No.                         4.11
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
"""